In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, regularizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.initializers import RandomNormal
from tensorflow.keras.optimizers import legacy
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from tqdm import tqdm

In [ ]:
features = ['A', 'd', 'L/d', 'fc', 'fyl', 'fys', 'ρl', 'ρs','γl','γs','n']
y_labels = ['yield_dis', 'yield_load', 'peak_dis', 'peak_load', 'ultimate_dis']

df = pd.read_csv('NOFRP01.csv', header=None,names=features + y_labels)

X = df[features].values
y = df[y_labels].values

In [ ]:
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=80)

In [ ]:
model = Sequential()
model.add(Dense(15, input_dim=11, activation='relu', kernel_initializer=RandomNormal())) 
model.add(Dense(9, activation='relu', kernel_initializer=RandomNormal()))
model.add(Dense(5, activation='relu'))

In [ ]:
opt = legacy.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss='mean_absolute_error', metrics=['mean_squared_error'])

In [ ]:
model.fit(X_train_tf, y_train_tf, epochs=300, batch_size=50)

In [ ]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r_squared = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)
print("R2:",r_squared)
print("Mean Absolute Percentage Error:", mape)

In [ ]:
y_ot = scaler_y.inverse_transform(y_test)
y_op = scaler_y.inverse_transform(y_pred)

mse_per_neuron = [mean_squared_error(y_ot[:, i], y_op[:, i]) for i in range(y_ot.shape[1])]
mae_per_neuron = [mean_absolute_error(y_ot[:, i], y_op[:, i]) for i in range(y_ot.shape[1])]
rmse_per_neuron = [np.sqrt(mse) for mse in mse_per_neuron]
r2_per_neuron = [r2_score(y_ot[:, i],y_op[:, i]) for i in range(y_ot.shape[1])]
mape_per_neuron = [mean_absolute_percentage_error(y_ot[:, i], y_op[:, i]) for i in range(y_ot.shape[1])]

error_df = pd.DataFrame({'Neuron': y_labels, 'MAE': mae_per_neuron , 'RMSE': rmse_per_neuron, 'R2': r2_per_neuron, 'MAPE': mape_per_neuron})

display(error_df)

In [ ]:
df = pd.DataFrame(y_ot)
df.to_excel('/Users/liuge/Desktop/Data/outputpred11.xlsx', index=False)
df = pd.DataFrame(y_op)
df.to_excel('/Users/liuge/Desktop/Data/outputpred22.xlsx', index=False)

In [ ]:
y_pred0 = model.predict(X_train)

mse = mean_squared_error(y_train, y_pred0)
mae = mean_absolute_error(y_train, y_pred0)
rmse = np.sqrt(mse)
r_squared = r2_score(y_train, y_pred0)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)
print("R2:",r_squared)
print("Mean Absolute Error:", mape)

In [ ]:
y_ot1 = scaler_y.inverse_transform(y_train)
y_op1 = scaler_y.inverse_transform(y_pred0)

mse_per_neuron = [mean_squared_error(y_ot1[:, i], y_op1[:, i]) for i in range(y_ot1.shape[1])]
mae_per_neuron = [mean_absolute_error(y_ot1[:, i], y_op1[:, i]) for i in range(y_ot1.shape[1])]
rmse_per_neuron = [np.sqrt(mse) for mse in mse_per_neuron]
r2_per_neuron = [r2_score(y_ot1[:, i],y_op1[:, i]) for i in range(y_ot1.shape[1])]
mape_per_neuron = [mean_absolute_percentage_error(y_ot1[:, i], y_op1[:, i]) for i in range(y_ot1.shape[1])]

error_df = pd.DataFrame({'Neuron': y_labels, 'MAE': mae_per_neuron , 'RMSE': rmse_per_neuron, 'R2': r2_per_neuron, 'MAPE': mape_per_neuron})

display(error_df)

In [ ]:
df = pd.DataFrame(y_ot1)
df.to_excel('/Users/liuge/Desktop/Data/outputpred33.xlsx', index=False)
df = pd.DataFrame(y_op1)
df.to_excel('/Users/liuge/Desktop/Data/outputpred44.xlsx', index=False)

In [ ]:
# 获取模型对应的权重进行优化
weights = model.get_weights()
last_layer_weights = weights[-2]
print("Last Layer Weights Shape:", last_layer_weights.shape)
neurons_to_optimize = last_layer_weights[:, 0]
print(neurons_to_optimize.shape)

# 定义萤火虫算法的参数
num_fireflies = 400
num_iterations = 500
alpha = 0.7 # 随机参数
beta_min = 0.5
gamma = 1 # 吸引度

In [ ]:
fireflies = np.random.randn(num_fireflies, len(neurons_to_optimize))

# 定义计算误差的函数
def compute_error(predictions, true_values):
    return np.mean((predictions - true_values) ** 2)

def evaluate_firefly_brightness(weights_fixed, firefly, inputs, outputs):
    layer_index = -2

    test_weights = weights_fixed.copy()

    test_weights[layer_index][:, 0] = firefly

    model.set_weights(test_weights)

    predictions = model.predict(inputs)

    loss = compute_error(predictions, outputs)

    return 1 / (loss + 1e-10)

In [ ]:
inputs, outputs = X_train, y_train

for iteration in range(num_iterations):
    brightness = np.zeros(num_fireflies)
    for i in range(num_fireflies):
        brightness[i] = evaluate_firefly_brightness(weights, fireflies[i], inputs, outputs)

    sorted_indices = np.argsort(brightness)
    fireflies = fireflies[sorted_indices]
    brightness = brightness[sorted_indices]

    for i in range(num_fireflies):
        for j in range(num_fireflies):
            if brightness[j] > brightness[i]:
                distance = np.linalg.norm(fireflies[i] - fireflies[j])
                beta = beta_min + (1 - beta_min) * np.exp(-gamma * distance ** 2)
                fireflies[i] = fireflies[i] * (1 - beta) + fireflies[j] * beta + alpha * (np.random.rand(len(neurons_to_optimize)) - 0.5)

best_weights = fireflies[0]

weights[-2][:, 0] = best_weights
model.set_weights(weights)

In [ ]:
y_pred1 = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred1)
mse = mean_squared_error(y_test, y_pred1)
rmse = np.sqrt(mse)
r_squared = r2_score(y_test, y_pred1)

print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)
print("R2:",r_squared)

In [ ]:
y_ot = scaler_y.inverse_transform(y_test)
y_op01 = scaler_y.inverse_transform(y_pred1)

mse_per_neuron = [mean_squared_error(y_ot[:, i], y_op01[:, i]) for i in range(y_ot.shape[1])]
mae_per_neuron = [mean_absolute_error(y_ot[:, i], y_op01[:, i]) for i in range(y_ot.shape[1])]
rmse_per_neuron = [np.sqrt(mse) for mse in mse_per_neuron]
r2_per_neuron = [r2_score(y_ot[:, i],y_op01[:, i]) for i in range(y_ot.shape[1])]

error_df = pd.DataFrame({'Neuron': y_labels, 'MAE': mae_per_neuron , 'RMSE': rmse_per_neuron, 'R2': r2_per_neuron})

display(error_df)

In [ ]:
y_pred0 = model.predict(X_train)

mse = mean_squared_error(y_train, y_pred0)
mae = mean_absolute_error(y_train, y_pred0)
rmse = np.sqrt(mse)
r_squared = r2_score(y_train, y_pred0)

print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)
print("R2:",r_squared)

In [ ]:
y_ot1 = scaler_y.inverse_transform(y_train)
y_op1 = scaler_y.inverse_transform(y_pred0)

mse_per_neuron = [mean_squared_error(y_ot1[:, i], y_op1[:, i]) for i in range(y_ot1.shape[1])]
mae_per_neuron = [mean_absolute_error(y_ot1[:, i], y_op1[:, i]) for i in range(y_ot1.shape[1])]
rmse_per_neuron = [np.sqrt(mse) for mse in mse_per_neuron]
r2_per_neuron = [r2_score(y_ot1[:, i],y_op1[:, i]) for i in range(y_ot1.shape[1])]

error_df = pd.DataFrame({'Neuron': y_labels, 'MAE': mae_per_neuron , 'RMSE': rmse_per_neuron, 'R2': r2_per_neuron})

display(error_df)

In [ ]:
results = []
print('Computing feature importance...')

baseline_mae = mae
results.append({'feature':'BASELINE', 'mae':baseline_mae})

for k in tqdm(range(X_scaled.shape[1])):
    save_col = X_scaled[:,k].copy()
    np.random.shuffle(X_scaled[:,k])

    y_pred_shuffled = model.predict(X_scaled)
    mae = mean_absolute_error(y_scaled[:, 0], y_pred_shuffled[:, 0])
    results.append({'feature': features[k], 'mae': mae})

    X_scaled[:,k] = save_col

print()
df = pd.DataFrame(results)
df = df.sort_values('mae')
plt.figure(figsize=(10, 6))
plt.barh(df.feature, df.mae)
plt.title('Feature Importance', size=16)
plt.xlabel('MAE of yield_dis')
plt.savefig("/Users/liuge/Desktop/my_plot1.png")
plt.show()

df.to_csv('feature_importance1.csv', index=False)

print(df)

In [ ]:
results = []
print('Computing feature importance...')

baseline_mae = mae
results.append({'feature':'BASELINE', 'mae':baseline_mae})

for k in tqdm(range(X_scaled.shape[1])):
    save_col = X_scaled[:,k].copy()
    np.random.shuffle(X_scaled[:,k])

    y_pred0_shuffled = model.predict(X_scaled)
    mae = mean_absolute_error(y_scaled[:, 1], y_pred0_shuffled[:, 1])
    results.append({'feature': features[k], 'mae': mae})

    X_scaled[:,k] = save_col

print()
df = pd.DataFrame(results)
df = df.sort_values('mae')
plt.figure(figsize=(10, 6))
plt.barh(df.feature, df.mae)
plt.title('Feature Importance', size=16)
plt.xlabel('MAE of yield_load')
plt.savefig("/Users/liuge/Desktop/my_plot2.png")
plt.show()

df.to_csv('feature_importance2.csv', index=False)

print(df)

In [ ]:
results = []
print('Computing feature importance...')

baseline_mae = mae
results.append({'feature':'BASELINE', 'mae':baseline_mae})

for k in tqdm(range(X_scaled.shape[1])):
    save_col = X_scaled[:,k].copy()
    np.random.shuffle(X_scaled[:,k])

    y_pred_shuffled = model.predict(X_scaled)
    mae = mean_absolute_error(y_scaled[:, 2], y_pred_shuffled[:, 2])
    results.append({'feature': features[k], 'mae': mae})

    X_scaled[:,k] = save_col

print()
df = pd.DataFrame(results)
df = df.sort_values('mae')
plt.figure(figsize=(10, 6))
plt.barh(df.feature, df.mae)
plt.title('Feature Importance', size=16)
plt.xlabel('MAE of peak_dis')
plt.savefig("/Users/liuge/Desktop/my_plot3.png")
plt.show()

df.to_csv('feature_importance3.csv', index=False)

print(df)

In [ ]:
results = []
print('Computing feature importance...')

baseline_mae = mae
results.append({'feature':'BASELINE', 'mae':baseline_mae})

for k in tqdm(range(X_scaled.shape[1])):
    save_col = X_scaled[:,k].copy()
    np.random.shuffle(X_scaled[:,k])

    y_pred_shuffled = model.predict(X_scaled)
    mae = mean_absolute_error(y_scaled[:, 3], y_pred_shuffled[:, 3])
    results.append({'feature': features[k], 'mae': mae})

    X_scaled[:,k] = save_col

print()
df = pd.DataFrame(results)
df = df.sort_values('mae')
plt.figure(figsize=(10, 6))
plt.barh(df.feature, df.mae)
plt.title('Feature Importance', size=16)
plt.xlabel('MAE of peak_load')
plt.savefig("/Users/liuge/Desktop/my_plot4.png")
plt.show()

df.to_csv('feature_importance4.csv', index=False)

print(df)

In [ ]:
results = []
print('Computing feature importance...')

baseline_mae = mae
results.append({'feature':'BASELINE', 'mae':baseline_mae})

for k in tqdm(range(X_scaled.shape[1])):
    save_col = X_scaled[:,k].copy()
    np.random.shuffle(X_scaled[:,k])

    y_pred_shuffled = model.predict(X_scaled)
    mae = mean_absolute_error(y_scaled[:, 4], y_pred_shuffled[:, 4])
    results.append({'feature': features[k], 'mae': mae})

    X_scaled[:,k] = save_col

print()
df = pd.DataFrame(results)
df = df.sort_values('mae')
plt.figure(figsize=(10, 6))
plt.barh(df.feature, df.mae)
plt.title('Feature Importance', size=16)
plt.xlabel('MAE of ultimate_dis')
plt.savefig("/Users/liuge/Desktop/my_plot5.png")
plt.show()

df.to_csv('feature_importance5.csv', index=False)

print(df)